In [9]:
import aiohttp
import asyncio
import json
import os
from typing import Dict
from openai import OpenAI

def get_prompt(query: str) -> str:
    """Return the hardcoded prompt template with the user's query"""
    return f"""You are an expert at translating a user query for company search (natural language) into a Companies API /companies API call.

The API call consists of the following:
- attribute
- operator
- sign
- values

The attributes must match one of the following:
- about.businessType
- about.industries
- about.industry
- about.totalEmployees
- about.yearFounded
- finances.revenue
- finances.stockExchange
- locations.headquarters.city.code
- locations.headquarters.continent.code
- locations.headquarters.country.code
- locations.headquarters.state.code
- socials
- technologies.active
- technologies.categories

The operator must always be 'and'

The sign must match one of the following:
- equals
- notEquals

The values must be an array of strings which match the attribute.

Notes:
1. For businessType, values must be one of:
   ["educational-institution", "nonprofit", "government-agency", "partnership", "privately-held", "public-company", "self-employed", "sole-proprietorship"]

2. For industries, if the industry has two words, separate them with a hyphen.

3. For totalEmployees, values must be one of:
   ["1-10", "10-50", "50-200", "200-500", "500-1k", "1k-5k", "5k-10k", "over-10k"]
   If the user query doesn't fit exactly, you can add multiple ranges.

4. For location.headquarters.city.code format:
   "city-code|state-code|country-code"
   Example: "new-york|new-york|us"

5. For location.headquarters.state.code format:
   "state-code|country-code"
   Example: "new-york|us"

6. For location.headquarters.country.code format:
   ["country-code"]
   Example: ["us"]

User Query: {query}

Return only a JSON array of search parameters. Example format:
[
    {{
        "attribute": "about.industries",
        "operator": "and",
        "sign": "equals",
        "values": ["marketing"]
    }},
    {{
        "attribute": "locations.headquarters.city.code",
        "operator": "and",
        "sign": "equals",
        "values": ["new-york|new-york|us"]
    }},
    {{
        "attribute": "about.totalEmployees",
        "operator": "and",
        "sign": "equals",
        "values": ["200-500"]
    }}
]"""



async def test_industries_endpoint() -> Dict:
    """Test the companies API industries endpoint"""
    try:
        api_key = os.getenv('COMPANIES_API_KEY')
        if not api_key:
            raise ValueError("COMPANIES_API_KEY environment variable not set")

        async with aiohttp.ClientSession() as session:
            headers = {'Authorization': f'Bearer {api_key}'}
            url = 'https://api.thecompaniesapi.com/v2/industries'

            async with session.get(url, headers=headers) as response:
                if response.status == 200:
                    data = await response.json()
                    print(json.dumps(data, indent=2))
                    return data
                else:
                    print(f"Error: {response.status}")
                    return {}

    except Exception as e:
        print(f"Error: {str(e)}")
        return {}

# Test both function

In [14]:
from typing import List


async def get_search_params(query: str) -> Dict:
    """Get search parameters from ChatGPT"""
    try:
        client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a company search query expert."},
                {"role": "user", "content": get_prompt(query)}
            ]
        )
        result = response.choices[0].message.content
        
        # Clean the response
        result = result.strip()
        if result.startswith('```json'):
            result = result[7:]
        if result.startswith('```'):
            result = result[3:]
        if result.endswith('```'):
            result = result[:-3]
        result = result.strip()
        
        return json.loads(result)
        
    except Exception as e:
        print(f"Error getting search params: {str(e)}")
        return {}
    

query = "Find biotech companies in nj that have 100-500 employees"
search_params = await get_search_params(query)
print(json.dumps(search_params, indent=2))


async def _search_companies(self, search_params: List[Dict]) -> List[Dict]:
    """
    Search companies using the Companies API and analyze with GPT-4
    """
    try:
        # Call Companies API
        async with aiohttp.ClientSession() as session:
            headers = {'Authorization': f'Basic {self.companies_api_key}'}
            url = 'https://api.thecompaniesapi.com/v2/companies/'
            params = {'query': json.dumps(search_params),
                      'page': 1,
                      'size': 3}

            async with session.get(url, headers=headers, params=params) as response:
                if response.status != 200:
                    print(f"Companies API error: {response.status}")
                    return []
                companies_data = await response.json()
                print(f"Found companies: {companies_data}")
        return companies_data
    
    except Exception as e:
        print(f"Error in _search_companies: {str(e)}")
        return []
    
# Create instance of class since _search_companies is an instance method
class CompanySearcher:
    def __init__(self):
        self.companies_api_key = os.getenv('COMPANIES_API_KEY')

searcher = CompanySearcher()

search_results = await _search_companies(search_params)
print(json.dumps(search_results, indent=2))




[
  {
    "attribute": "about.industries",
    "operator": "and",
    "sign": "equals",
    "values": [
      "biotechnology"
    ]
  },
  {
    "attribute": "locations.headquarters.state.code",
    "operator": "and",
    "sign": "equals",
    "values": [
      "new-jersey|us"
    ]
  },
  {
    "attribute": "about.totalEmployees",
    "operator": "and",
    "sign": "equals",
    "values": [
      "100-200",
      "200-500"
    ]
  }
]


/Users/shivan/Library/Python/3.9/lib/python/site-packages/executing/executing.py:297: RuntimeWarning: coroutine 'test_industries_endpoint' was never awaited
  result = cls.__dict__.get(name, default)


TypeError: _search_companies() missing 1 required positional argument: 'search_params'

In [ ]:
import aiohttp
import asyncio
import json
import os
from typing import Dict

async def test_industries_endpoint() -> Dict:
    """Test the companies API industries endpoint"""
    try:
        api_key = os.getenv('COMPANIES_API_KEY')
        if not api_key:
            raise ValueError("COMPANIES_API_KEY environment variable not set")

        async with aiohttp.ClientSession() as session:
            headers = {'Authorization': f'Basic {api_key}'}
            url = 'https://api.thecompaniesapi.com/v2/companies?&size=2'

            async with session.get(url, headers=headers) as response:
                if response.status == 200:
                    data = await response.json()
                    print(json.dumps(data, indent=2))
                    return data
                else:
                    print(f"Error: {response.status}")
                    return {}

    except Exception as e:
        print(f"Error: {str(e)}")
        return {}

# Run the test using asyncio event loop that works in Jupyter
await test_industries_endpoint()

In [6]:
from openai import OpenAI
import os
from typing import Dict

async def generate_intent_signals(user_input: str) -> Dict:
    """Generate intent signals and search queries using Claude"""
    try:
        openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        response = await openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": """You are an expert at identifying B2B Sales signals and generating creative search queries.
                The user will provide a query and you will generate a list of companies that are showing intent signals that are relevant to the query.
                Your task is to:
                1. Generate a diverse list of intent signals that indicate a company might be interested in a product/service based on the user's query.
                2. Generate 25 creative SERP search queries to find companies showing these intent signals
            
                Return a single JSON with two arrays:
                1. intent_signals: Array of intent signal strings
                2. search_queries: Array of 25 search queries to find these signals"""},
                {"role": "user", "content": user_input}
            ]
        )
        
        result = json.loads(response.choices[0].message.content)
        print(result)
        return result
        
    except Exception as e:
        print(f"Error generating intent signals: {str(e)}")
        raise

response = generate_intent_signals("Find biotech companies in nj that have 100-500 employees investing in AI")
await response



Error generating intent signals: object ChatCompletion can't be used in 'await' expression


TypeError: object ChatCompletion can't be used in 'await' expression

In [7]:
import secrets

print(secrets.token_hex(32))

fc5c89918811307a28fe5be1840f2a7866050244d78179bf443576365357db8c
